# Assignment 2: Decision Trees

In this assignment, you are going to implement a decision tree (or random forest) to forcast the weather.

## Description

- You must implement a `Model` class for training and predicting:
  ```python
  X, y = load_dataset()

  model = Model(num_features, num_classes)

  # training
  model.fit(X, y)

  # prediction
  y_pred = model.predict(X)
  ```
- Please search (Ctrl+F) for `TODO` to see what you need to do.
- About the dataset
  - Given the **training set**, please **train/validate** on it.  
  (note that your model will get bad testing score if it overfits on the training set)
  - After submitting the assignment, we will train on the same training set and test on the hidden **testing set** for scoring (using [f1-score](https://towardsdatascience.com/a-look-at-precision-recall-and-f1-score-36b5fd0dd3ec#11b8)).

### Typical performance

- **Random Guess**  
  F1-Score: 0.30  
  Accuracy: 0.50
- **Always Predict 1**  
  F1-Score: 0.37  
  Accuracy: 0.22
- **Always Predict 0**  
  F1-Score: 0.00  
  Accuracy: 0.77
- **sklearn.tree.DecisionTreeClassifier**  
  - **Training (5-fold cross-validation mean)**  
    F1-Score: 0.63-0.99  
    Accuracy: 0.85-0.99
  - **Validation (5-fold cross-validation mean)**  
    F1-Score: 0.50-0.60  
    Accuracy: 0.75-0.90


In [1]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

import os
import urllib.request
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score


def load_dataset(url):
  """ Get and load weather dataset. """

  path = url.split('/')[-1] # get the file name from url

  if not os.path.exists(path):
    print('Download:', url)
    urllib.request.urlretrieve(url, path)

  return pd.read_pickle(path) # pickle protocol=4


def get_input_target(df):
  """ Get X and y from weather dataset. """
  
  target_column = 'RainTomorrow' # predict 1 of it rains tomorrow

  X = df.drop(columns=[target_column]).to_numpy()
  y = df[target_column].to_numpy()

  return X, y


def k_fold_cv(model_create_fn, X, y, k=5):
  """ Run k-fold cross-validation. """

  results = []

  idxs = list(range(X.shape[0]))
  np.random.shuffle(idxs)

  for i, (train_idxs, val_idxs) in enumerate(KFold(k).split(idxs)):
    splits = {'train': (X[train_idxs], y[train_idxs]),
              'val':   (X[val_idxs],   y[val_idxs]  )}

    print('Run {}:'.format(i+1))

    model = model_create_fn()
    model.fit(*splits['train']) # training

    for name, (X_split, y_split) in splits.items():
      y_pred = model.predict(X_split)

      result = {'split': name,
                'f1': f1_score(y_pred, y_split),
                'acc': accuracy_score(y_pred, y_split)}
      results.append(result)

      print('{split:>8s}: f1={f1:.4f} acc={acc:.4f}'.format(**result))

  return pd.DataFrame(results)


# @begin

In [2]:
# Decision Tree's Node
class Node:
    leaf = None        # bool  : is leaf or not
    result = None      # int   : predict result, 1 -> rain tomorrow, 0 -> not
    f_col = None       # int   : feature column
    f_num = None       # float : feature reference
    lef_chi = None     # Node  : <= ref
    rig_chi = None     # Node  : > ref

    def __init__(self, leaf: bool, result: int, f_col: int, f_num: float):
        self.leaf = leaf
        self.result = result
        self.f_col = f_col
        self.f_num = f_num
        self.lef_chi = None
        self.rig_chi = None

    def putChild(self, lef_chi: "Node", rig_chi: "Node"):
        self.leaf = 0
        self.lef_chi = lef_chi
        self.rig_chi = rig_chi

    def calResult(self, y: np.ndarray):
        self.leaf = 1
        cnt = 0
        for i in y:
            if i == 1:
                cnt += 1
        if cnt > len(y) - cnt:
            self.result = 1
        else:
            self.result = 0

    def isLeaf(self) -> bool:
        return self.leaf
    
    def getCol(self) -> int:
        return self.f_col
    
    def getNum(self) -> float:
        return self.f_num

    def getResult(self) -> int:
        return self.result

    def walkDown(self, X: np.ndarray) -> "Node":
        if (X[self.f_col] <= f_num):
            return self.lef_chi;
        return self.rig_chi;

In [3]:
def extractRain(X: np.ndarray, y: np.ndarray) -> [np.ndarray]:
    xx = [] # rain
    yy = [] # not rain
    for i in range(len(y_train)):
        if (y_train[i] == 1):
            xx.append(X[i])
        else:
            yy.append(X[i])
    return [xx, yy]

def _extractRaintest():
    tmp1, tmp2 = extractRain(X_train, y_train)

    print(tmp1[0], "\nRain Total Number: ", len(tmp1))
    print(tmp2[0], "\nNot Rain Total Number: ", len(tmp2))

    print("\nTotal trainning cases: ", len(y_train))

    # p = np.random.randint(0, 1, size=10)
    # print(p)

In [4]:
def calPurity(X: np.ndarray, y: np.ndarray, j: int, now: float) -> float:
    n = len(y)

    sml_cnt = 0
    sml_ans = 0
    big_cnt = 0
    big_ans = 0
    for i in range(n):
        if X[i][j] <= now:
            sml_cnt += 1
            sml_ans += (y[i] == 1)
        else:
            big_cnt += 1
            big_ans += (y[i] == 1)
    
    if (sml_cnt == 0 or big_cnt == 0):
        return -1000

    ans = max(((sml_cnt-sml_ans) / sml_cnt) + (big_ans / big_cnt), 
              ((big_cnt-big_ans) / big_cnt) + (sml_ans / sml_cnt),)
    return ans

In [5]:
def getBiggestFeat(X: np.ndarray, y: np.ndarray, deb: int) -> (int, float):
    f_ret = 0
    f_num = 0.0
    f_now = 0.0

    if len(X) <= 1:
        return (0, 0)

    n = len(y)
    m = len(X[0])
    for i in range(m):
        now = 0
        sum = 0
        cnt = 0
        for j in range(n):
            # if (y[j] == 1):
                # now += X[j][i]
                # cnt += 1
            sum += X[j][i]
        
        # sum = sum / n
        # now = now / n
        # now = (sum + now) / 2.0
        
        now = sum / n

        prt = calPurity(X, y, i, now)
        if prt > f_now:
            f_now = prt
            f_ret = i
            f_num = now
        
        if (deb):
            print(i, prt)

    return (f_ret, f_num)

# print(getBiggestFeat(X_train, y_train, 1))

In [6]:
def dfs(X: np.ndarray, y: np.ndarray, nod: Node, dep: int):
    lef_x = []
    lef_y = []
    rig_x = []
    rig_y = []

    p = nod.getCol()
    f = nod.getNum()
    for i in range(len(y)):
        if X[i][p] <= f:
            lef_x.append(X[i])
            lef_y.append(y[i])
        else:
            rig_x.append(X[i])
            rig_y.append(y[i])
    
    lef_p, lef_n = getBiggestFeat(lef_x, lef_y, 0)
    rig_p, rig_n = getBiggestFeat(rig_x, rig_y, 0)

    lefChd = Node(dep == 0, 1, lef_p, lef_n)
    rigChd = Node(dep == 0, 1, rig_p, rig_n)
    
    nod.putChild(lefChd, rigChd)

    if (dep == 0): 
        lefChd.calResult(lef_y)
        rigChd.calResult(rig_y)
        return

    if (len(lef_x) <= 1): lefChd.calResult(lef_y)
    else:
        dfs(lef_x, lef_y, lefChd, dep - 1)
        
    if (len(rig_x) <= 0): rigChd.calResult(rig_y)
    else:
        dfs(rig_x, rig_y, rigChd, dep - 1)

    return

In [7]:
# TODO: you can define or import something here (optional)

class Model:
    num_depth = 8
    num_features = 0
    num_classes = 0
    rot = None

    """ Initialize the model.
    Args:
        num_features (int) : the input feature size.
        num_classes (int) : number of output classes.
    """
    def __init__(self, num_features: int, num_classes: int):  # TODO: implement your model initialization here (optional)
        self.num_features = num_features
        self.num_classes = num_classes
        self.rot = None

    """ Train on input/target pairs.
    Args:
        X (np.ndarray) : training inputs with shape (num_inputs, num_features).
        y (np.ndarray) : training targets with shape (num_inputs,).
    """
    def fit(self, X: np.ndarray, y: np.ndarray):  # TODO: implement your training algorithm here 
        fet_poi, fet_num = getBiggestFeat(X, y, 0)
        self.rot = Node(0, 1, fet_poi, fet_num)

        dfs(X, y, self.rot, self.num_depth)

        return


    """ Predict y given X.
    Args:
        X (np.ndarray) : inputs, shape: (num_inputs, num_features).
    Returns:
        np.ndarray : the predicted integer outputs, shape: (num_inputs,).
    """
    def predict(self, X: np.ndarray) -> np.ndarray: # TODO: implement your prediction algorithm here
        # p = np.random.randint(0, self.num_classes, size=X.shape[0]) # (delete this)
        
        p = []
        n = X.shape[0]
        for i in X:
            now = self.rot
            while now.isLeaf() != 1:
                poi = now.getCol()
                num = now.getNum()
                if i[poi] <= num:
                    now = now.lef_chi
                else:
                    now = now.rig_chi

            p.append(now.getResult())

        return p

In [8]:
# @end

###########################
# DO NOT CHANGE THIS CELL #
###########################

df = load_dataset('https://lab.djosix.com/weather.pkl')
X_train, y_train = get_input_target(df)

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,RainToday,RainTomorrow
0,7.4,25.100000,0.0,5.469824,7.624853,44.0,4.0,22.0,44.0,25.0,1010.599976,4.43719,4.503167,17.200001,0.0,0
1,12.9,25.700001,0.0,5.469824,7.624853,46.0,19.0,26.0,38.0,30.0,1007.599976,4.43719,2.000000,21.000000,0.0,0
2,9.2,28.000000,0.0,5.469824,7.624853,24.0,11.0,9.0,45.0,16.0,1017.599976,4.43719,4.503167,18.100000,0.0,0
3,17.5,32.299999,1.0,5.469824,7.624853,41.0,7.0,20.0,82.0,33.0,1010.799988,7.00000,8.000000,17.799999,0.0,0
4,14.6,29.700001,0.2,5.469824,7.624853,56.0,19.0,24.0,55.0,23.0,1009.200012,4.43719,4.503167,20.600000,0.0,0


In [9]:
def _lookData():
    # X_train[i][2] -> rainfall
    cnt = 0
    k = 0
    for i in range(len(y_train)):
        if (y_train[i] == 1):
            cnt += 1
            if (X_train[i][2] > 0):
                k += 1

    print("Total Rain Fall ->", cnt)
    print("Rainfall before today rain -> ", k, ", Percent Tage:", k / cnt)

In [10]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

create_model = lambda: Model(X_train.shape[1], 2)
k_fold_cv(create_model, X_train, y_train).groupby('split').mean()

Run 1:
   train: f1=0.5718 acc=0.8431
     val: f1=0.5251 acc=0.8370
Run 2:
   train: f1=0.5848 acc=0.8465
     val: f1=0.5627 acc=0.8186
Run 3:
   train: f1=0.5856 acc=0.8451
     val: f1=0.5709 acc=0.8259
Run 4:
   train: f1=0.5680 acc=0.8440
     val: f1=0.5478 acc=0.8306
Run 5:
   train: f1=0.5713 acc=0.8395
     val: f1=0.5314 acc=0.8524


,f1,acc
split,,
train,0.576312,0.843625
val,0.547580,0.832914


## Submission

1. Make sure your code runs correctly after clicking `"Runtime" > "Restart and run all"`
2. Rename this notebook to `XXXXXXX_2.ipynb`, where `XXXXXXX` is your student ID.
3. Download IPython notebook: `"File" > "Download" > "Download .ipynb"`
4. Download Python source code: `"File" > "Download" > "Download .py"`
5. Create a zip file for `XXXXXXX_2.ipynb` and `XXXXXXX_2.py`  
   named `XXXXXXX_2.zip`, where `XXXXXXX` is your student ID.
6. Upload the zip file to E3.

😊 Good luck!